In [11]:
import os
import numpy as np
import requests
import concurrent.futures
import pandas as pd
import decord

In [ ]:
def read_frames_decord(video_path):
    video_reader = decord.VideoReader(video_path, num_threads=1)
    return

In [12]:
def request_save(url, save_fp):
    try:
            if os.path.isfile(save_fp) and os.path.getsize(save_fp)>100000:
                # imgs, idxs = video_reader(video_fp)
                pass
            else:
                img_data = requests.get(url, timeout=5).content
                with open(save_fp, 'wb') as handler:
                    handler.write(img_data)
    except Exception as e:
        raise ValueError(
            f'Video loading failed for {save_fp}, video loading for this dataset is strict.') from e
        
    

In [13]:
# parameters
class MyArgs():
    def __init__(self) -> None:
        self.partitions = 1
        self.part = 0
        self.data_dir = 'data/WebVid'
        self.csv_path ='data/WebVid/release/results_2M_val.csv'
        self.processes = 8

In [14]:
args = MyArgs()
# split dataset
### preproc
video_dir = os.path.join(args.data_dir, 'videos')
if not os.path.exists(os.path.join(video_dir, 'videos')):
    os.makedirs(os.path.join(video_dir, 'videos'))

# ASSUMES THE CSV FILE HAS BEEN SPLIT INTO N PARTS
partition_dir = args.csv_path.replace('.csv', f'_{args.partitions}')
# if not, then split in this job.
if not os.path.exists(partition_dir):
    os.makedirs(partition_dir)
    full_df = pd.read_csv(args.csv_path)
    df_split = np.array_split(full_df, args.partitions)
    for idx, subdf in enumerate(df_split):
        subdf.to_csv(os.path.join(partition_dir, f'{idx}.csv'), index=False)

relevant_fp = os.path.join(args.data_dir, 'relevant_videos_exists.txt')
if os.path.isfile(relevant_fp):
    exists = pd.read_csv(os.path.join(args.data_dir, 'relevant_videos_exists.txt'), names=['fn'])
else:
    exists = []
len(exists)

0

In [15]:
# df = pd.read_csv(os.path.join(partition_dir, f'{args.part}.csv'))
df = pd.read_csv(args.csv_path)
l = len(df)
# df[df.isnull().values==True]
df = df.dropna(axis=0,how='any')
l_a = len(df)
print(l_a, l - l_a)

4998 1


In [16]:
df['rel_fn'] = df.apply(lambda x: os.path.join(x['page_dir'], str(x['videoid'])), axis=1)
df['rel_fn'] = df['rel_fn'] + '.mp4'
df = df[~df['rel_fn'].isin(exists)]
df.head()

,videoid,name,page_idx,page_dir,duration,contentUrl,rel_fn
0,1053841541,Travel blogger shoot a story on top of mountai...,27732,027701_027750,PT00H00M17S,https://ak.picdn.net/shutterstock/videos/10538...,027701_027750/1053841541.mp4
1,5641526,Horse grazing - seperated on green screen,28620,054101_054150,PT00H00M24S,https://ak.picdn.net/shutterstock/videos/56415...,054101_054150/5641526.mp4
2,1019039938,City traffic lights. blurred view,22404,022401_022450,PT00H00M13S,https://ak.picdn.net/shutterstock/videos/10190...,022401_022450/1019039938.mp4
3,6251156,Asian girl sexy santa claus isolated greenscre...,4046,004001_004050,PT00H00M16S,https://ak.picdn.net/shutterstock/videos/62511...,004001_004050/6251156.mp4
4,23720764,Young woman flexing muscles with barbell in gy...,13617,013601_013650,PT00H00M24S,https://ak.picdn.net/shutterstock/videos/23720...,013601_013650/23720764.mp4


In [7]:
playlists_to_dl = np.sort(df['page_dir'].unique())
len(playlists_to_dl)

1864

In [10]:
for page_dir in playlists_to_dl:
    vid_dir_t = os.path.join(video_dir, page_dir)
    # print(vid_dir_t)
    # break
    pdf = df[df['page_dir'] == page_dir]
    if len(pdf) > 0:
        if not os.path.exists(vid_dir_t):
            os.makedirs(vid_dir_t)

        urls_todo = []
        save_fps = []

        for idx, row in pdf.iterrows():
            video_fp = os.path.join(vid_dir_t, str(row['videoid']) + '.mp4')

            urls_todo.append(row['contentUrl'])
            save_fps.append(video_fp)

        # print(f'Spawning {len(urls_todo)} jobs for page {page_dir}')
        with concurrent.futures.ThreadPoolExecutor(max_workers=args.processes) as executor:
            future_to_url = {executor.submit(request_save, url, fp) for url, fp in zip(urls_todo, save_fps)}

data/WebVid/videos/000001_000050
